# Variational monte carlo simulation for Hydrogen Atom

In [77]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [78]:
def wavefunc(pos,a):
    return np.exp(-a*np.linalg.norm(pos,axis=1))

def updateWalker(pos,N,a):
    dis = np.random.normal(0,d,(N,3)) #proposed displacement
    p = (wavefunc(pos+dis,a)/wavefunc(pos,a))**2
    rnd = np.random.rand(N)
    pos += (rnd<p)[:,None]*dis
    
def EL(pos,a):
    r = np.linalg.norm(pos,axis=1)
    return -1/r - a/2 * (a - 2/r)

def updateVarParams(R,EL,a,gamma):
    r = np.linalg.norm(R,axis=1)
    dEda = 2 * (np.mean(-r*EL)-EL.mean()*np.mean(-r))
    return a - gamma * dEda, dEda

In [80]:
N = 400 #Number of walkers
T = 26000 #Number of displacement attempts per walker
T0 = 4000 #Reject first T0 timesteps from averaging

#Variational parameters
a = 0.8

d = a/2 #mean step length
gamma = 1 #damped steepest descent parameter
epsilon = 1e-12 # tolerance on derivative 
itmax = 12; #max number of optimization iterations

dEda = 2; it = 0;
while((abs(dEda) >= epsilon) & (it<itmax)):
    r = np.random.uniform(-2*a,2*a,(N,3))
    R = np.empty((T*N,3))
    
    for t in range(0,T0):
        updateWalker(r,N,a)
        
    for t in range(0,T):
        updateWalker(r,N,a)
        R[t*N:(t+1)*N,:] = r
        
    Eloc = EL(R,a)
    E = np.mean(Eloc)
    VarE = np.var(Eloc)
    print('a = ',a,' <E> = ',E,' Var(EL) = ',VarE)
    a, dEda = updateVarParams(R,Eloc,a,gamma)
    it+=1
    


if it==itmax:
    print('Maximum number of iterations reached. No optimal variational parameter may be found up to the required precision')

a =  0.8  <E> =  -0.480095342728  Var(EL) =  0.025036089327
a =  1.00036561275  <E> =  -0.499999605848  Var(EL) =  1.28880572289e-07
a =  1.00000063127  <E> =  -0.500000000499  Var(EL) =  3.86820441192e-13
a =  0.999999999045  <E> =  -0.5  Var(EL) =  8.98112963382e-19
a =  1.0  <E> =  -0.5  Var(EL) =  1.52935178404e-24
a =  1.0  <E> =  -0.5  Var(EL) =  3.93354811611e-29
